In [2]:
import cv2
import numpy as np
import os
import time

In [3]:
labelsPath = "object_detection_classes_coco.txt"
Labels = open(labelsPath).read().strip().split("\n")

In [4]:
weightPath = "frozen_inference_graph.pb"
configPath = "mask_rcnn_inception_v2_coco_2018_01_28.pbtxt"

In [5]:
Colors = np.random.randint(100, 255, (len(Labels), 3))

In [7]:

net = cv2.dnn.readNetFromTensorflow(weightPath, configPath)
image = cv2.imread("image2.jpg")
H, W = image.shape[:2]
convertedImage = cv2.dnn.blobFromImage(image, swapRB=True, crop=False)
net.setInput(convertedImage)
(objects, masks) = net.forward(["detection_out_final", "detection_masks"])

In [9]:
for i in range(objects.shape[2]):
    classID = int(objects[0,0, i, 1])
    conf = objects[0,0,i,2]
    if conf > 0.84:
        print(Labels[classID])
        
        box = objects[0,0,i, 3:7] * np.array([W,H, W,H])
        startX, startY, endX, endY = box.astype("int")
        
        boxW = endX - startX
        boxH = endY - startY
        
        mask = masks[i, classID]
        mask = cv2.resize(mask, (boxW, boxH))
        mask = (mask > 0.4)
        
        roi = image[startY:endY, startX:endX]
        roi = roi[mask]
        
        color = Colors[classID]
        B,G,R = color
        ColorMask = ((0.4 * color) + (0.6 * roi)).astype("uint8")
        image[startY:endY, startX:endX][mask] = ColorMask
        
        cv2.rectangle(image, (startX, startY), (endX, endY), (int(B),int(G),int(R)), 2)
        text = Labels[classID] + "--" + str(conf)
        #cv2.putText(image, text, (startX, startY-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
cv2.imshow("Output", image)
cv2.waitKey(5000)


person
person
person
bicycle
bicycle


-1

## YOLO Object Detection

In [1]:
import cv2
import numpy as np
import os
import time

In [2]:
labelsPath = "coco.names"
Labels = open(labelsPath).read().strip().split("\n")

weightPath = "yolov3.weights"
configPath = "yolov3.cfg"

Colors = np.random.randint(100, 255, (len(Labels), 3))



In [3]:

net = cv2.dnn.readNetFromDarknet(configPath, weightPath)
image = cv2.imread("image2.jpg")
H, W = image.shape[:2]

In [4]:
layers = net.getLayerNames()
ucl = net.getUnconnectedOutLayers()
LayerName = []
for i in ucl:
    LayerName.append(layers[i[0]-1])

convertedImage = cv2.dnn.blobFromImage(image, 1/255, (416, 416), swapRB=True, crop=False)
net.setInput(convertedImage)
prediction = net.forward(LayerName)

In [5]:
for output in prediction:
    for detection in output:
        score = detection[5:]
        conf = np.max(score)
        classId = np.argmax(score)
        if conf > 0.97:
            print(Labels[classId])
            Cx,Cy,w,h = (detection[:4] * np.array([W,H,W,H])).astype("int")
            x = int(Cx - (w//2))
            y = int(Cy - (h//2))
            cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,0), 2)

while True:
    cv2.imshow("Image", image)
    if cv2.waitKey(1) == 13:
        break

person
person
person
person
bicycle
bicycle


In [ ]:
score = prediction[1][1][5:]

## Live Video Object Detection

In [1]:
import cv2
import numpy as np
import os

In [2]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat","bottle", "bus", "car", "cat", "chair", "cow", "diningtable",

           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
             "sofa", "train", "tvmonitor","smartphone","hand"]

In [3]:

weightPath = "MobileNetSSD_deploy.caffemodel"
configPath = "MobileNetSSD_deploy.prototxt.txt"

Colors = np.random.randint(100, 255, (len(CLASSES), 3))


net = cv2.dnn.readNetFromCaffe(configPath, weightPath)


In [4]:
cam = cv2.VideoCapture(0)
while True:
    _, frame = cam.read()
    H, W = frame.shape[:2]
    image = cv2.resize(frame, (300, 300))
    convertedImage = cv2.dnn.blobFromImage(image, 0.007843, (300, 300), 127.5)
    net.setInput(convertedImage)
    detection = net.forward()
    
    for i in range(detection.shape[2]):
        classID = int(detection[0,0, i, 1])
        conf = detection[0,0,i,2]
        if conf > 0.84:
            labels = CLASSES[int(classID)]
            
            color = Colors[classID]
            B,G,R = color
      
            box = detection[0,0,i, 3:7] * np.array([W,H, W,H])
            startX, startY, endX, endY = box.astype("int")
            cv2.rectangle(frame, (startX, startY), (endX, endY), (int(B),int(G),int(R)), 3)
            cv2.putText(frame, labels, (startX, startY-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (int(B),int(G),int(R)), 3)
    cv2.imshow("Image", frame)
    if cv2.waitKey(1) == 13:
        break
    
            
    
    